# Azure AI Agents with Model Context Protocol (MCP) Support - Python

ဒီ notebook က Azure AI Agents ကို Model Context Protocol (MCP) tools တွေနဲ့ Python မှာ ဘယ်လိုအသုံးပြုရမလဲဆိုတာကို ပြသထားပါတယ်။ ဒါဟာ keyless authentication ကို အသုံးပြုပြီး အပြင်ဘက် MCP servers (Microsoft Learn ကဲ့သို့သော) ကို အသုံးချနိုင်တဲ့ အရည်အသွေးမြင့် agent တစ်ခုကို ဖန်တီးပုံကို ပြသထားပါတယ်။


## လိုအပ်သော Python Packages များကို ထည့်သွင်းပါ

အရင်ဆုံး၊ လိုအပ်သော Python packages များကို ထည့်သွင်းရမည်ဖြစ်သည်။
- **azure-ai-projects**: အဓိက Azure AI Projects SDK
- **azure-ai-agents**: Agent များကို ဖန်တီးခြင်းနှင့် စီမံခန့်ခွဲခြင်းအတွက် Azure AI Agents SDK
- **azure-identity**: DefaultAzureCredential ကို အသုံးပြု၍ keyless authentication ပေးသည်
- **mcp**: Python အတွက် Model Context Protocol အကောင်အထည်ဖော်မှု


## Keyless Authentication Benefits

ဒီ notebook က **keyless authentication** ရဲ့ အကျိုးကျေးဇူးတွေကို ဖော်ပြထားပါတယ်၊ အဓိကအားဖြင့် အောက်ပါ အကျိုးကျေးဇူးတွေ ရရှိနိုင်ပါတယ်။
- ✅ **API key မလိုအပ်ခြင်း** - Azure identity-based authentication ကို အသုံးပြုထားသည်
- ✅ **လုံခြုံရေး မြှင့်တင်ခြင်း** - Code သို့မဟုတ် configuration ဖိုင်များတွင် လျှို့ဝှက်ချက်များ မသိမ်းဆည်းထားခြင်း
- ✅ **အလိုအလျောက် credential ပြောင်းလဲမှု** - Azure က credential lifecycle ကို စီမံခန့်ခွဲပေးသည်
- ✅ **Role-based access control** - Azure RBAC ကို အသုံးပြု၍ ခွင့်ပြုချက်များကို စိတ်ကြိုက် စီမံနိုင်ခြင်း
- ✅ **ပတ်ဝန်းကျင်များစွာအတွက် အထောက်အပံ့** - Development နှင့် Production အတွက် အဆင်ပြေစွာ လုပ်ဆောင်နိုင်ခြင်း

`DefaultAzureCredential` က အကောင်းဆုံး credential အရင်းအမြစ်ကို အလိုအလျောက် ရွေးချယ်ပေးပါသည်။
1. **Managed Identity** (Azure တွင် run လုပ်နေသောအခါ)
2. **Azure CLI** credentials (local development အတွင်း)
3. **Visual Studio** credentials
4. **Environment variables** (configure လုပ်ထားပါက)
5. **Interactive browser** authentication (fallback အနေဖြင့်)


## Keyless Authentication Setup

**Keyless Authentication အတွက် လိုအပ်ချက်များ**

### Local Development အတွက်:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure Environments အတွက်:
- Azure resource တွင် **System-assigned Managed Identity** ကို enable လုပ်ပါ
- Managed identity အတွက် သင့်တော်သော **RBAC roles** များကို ပေးပါ:
  - Azure OpenAI ကို အသုံးပြုရန် `Cognitive Services OpenAI User`
  - Azure AI Projects ကို အသုံးပြုရန် `AI Developer`

### Environment Variables (Optional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**API keys သို့မဟုတ် connection strings မလိုအပ်ပါ!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## လိုအပ်သော Library များ Import လုပ်ပါ

လိုအပ်သော Python module များကို Import လုပ်ပါ:
- **os, time**: Environment variable နှင့် အချိန်နှောင့်နှေးမှုများအတွက် သာမန် Python library များ
- **AIProjectClient**: Azure AI Projects အတွက် အဓိက client
- **DefaultAzureCredential**: Azure service များအတွက် keyless authentication
- **MCP-related classes**: MCP tools ဖန်တီးခြင်းနှင့် အတည်ပြုမှုများကို စီမံရန်အတွက်


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## MCP Server ဆက်တင်များကို ပြင်ဆင်ရန်

ပတ်ဝန်းကျင် အပြောင်းအလဲများနှင့် fallback defaults များကို အသုံးပြု၍ MCP server ကို ဆက်တင်ပါ။
- **MCP_SERVER_URL**: MCP server ၏ URL (မူရင်းအနေဖြင့် Microsoft Learn API ကို သတ်မှတ်ထားသည်)
- **MCP_SERVER_LABEL**: MCP server ကို ဖော်ပြရန် အသုံးပြုမည့် အမည် (မူရင်းအနေဖြင့် "mslearn" ကို သတ်မှတ်ထားသည်)

ဤနည်းလမ်းသည် ပတ်ဝန်းကျင်အမျိုးမျိုးတွင် အလွယ်တကူ ဆက်တင်ပြင်ဆင်နိုင်စေရန် အထောက်အကူပြုသည်။


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Azure AI Project Client ကို ဖန်တီးခြင်း (Keyless Authentication)

**Keyless authentication** ကို အသုံးပြု၍ Azure AI Project client ကို စတင်ဖန်တီးပါ:
- **endpoint**: Azure AI Foundry project endpoint URL
- **credential**: `DefaultAzureCredential()` ကို အသုံးပြု၍ လုံခြုံပြီး keyless authentication ကို အကောင်အထည်ဖော်ပါ
- **API key မလိုအပ်ပါ**: အကောင်းဆုံး credential ကို အလိုအလျောက် ရှာဖွေပြီး အသုံးပြုပါ

**Authentication Flow:**
1. Managed Identity ကို စစ်ဆေးပါ (Azure environment တွင်)
2. Azure CLI credentials ကို fallback အနေဖြင့် အသုံးပြုပါ (local development အတွက်)
3. လိုအပ်သည့် အခြား credential sources များကို အသုံးပြုပါ

ဤနည်းလမ်းသည် သင့် code တွင် API keys သို့မဟုတ် connection strings များကို စီမံခန့်ခွဲရန် လိုအပ်မှုကို ဖယ်ရှားပေးပါသည်။


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## MCP Tool ကိုဖန်တီးခြင်း

Microsoft Learn MCP server ကိုချိတ်ဆက်နိုင်သော MCP tool ကိုဖန်တီးပါ။
- **server_label**: MCP server အတွက်အမှတ်အသား
- **server_url**: MCP server ရဲ့ URL endpoint
- **allowed_tools**: အသုံးပြုနိုင်သော tools များကို ကန့်သတ်ရန် optional list (empty list သည် tools အားလုံးကို ခွင့်ပြုသည်)

ဒီ tool က agent ကို Microsoft Learn documentation နှင့် resources များကို access လုပ်နိုင်စေပါမည်။


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## အေးဂျင့်တစ်ဦး ဖန်တီးပြီး စကားဝိုင်းကို အကောင်အထည်ဖော်ခြင်း (Keyless Workflow)

ဤအပိုင်းတွင် **keyless agent workflow** အပြည့်အစုံကို ဖော်ပြထားသည် -

1. **AI အေးဂျင့်တစ်ဦး ဖန်တီးရန်**: GPT-4.1 nano မော်ဒယ်နှင့် MCP tools များဖြင့် အေးဂျင့်တစ်ဦးကို စတင်တည်ဆောက်ပါ။
2. **Thread တစ်ခု ဖန်တီးရန်**: ဆက်သွယ်မှုအတွက် စကားဝိုင်း thread တစ်ခုကို တည်ဆောက်ပါ။
3. **Message ပို့ရန်**: Azure OpenAI နှင့် OpenAI အကြားကွာခြားချက်များကို အေးဂျင့်ကို မေးမြန်းပါ။
4. **Tool အတည်ပြုမှုများကို ကိုင်တွယ်ရန်**: လိုအပ်သည့်အခါ MCP tool ခေါ်ဆိုမှုများကို အလိုအလျောက် အတည်ပြုပါ။
5. **အကောင်အထည်ဖော်မှုကို စောင့်ကြည့်ရန်**: အေးဂျင့်၏ တိုးတက်မှုကို စောင့်ကြည့်ပြီး လိုအပ်သည့် လုပ်ဆောင်မှုများကို ကိုင်တွယ်ပါ။
6. **ရလဒ်များကို ပြသရန်**: စကားဝိုင်းနှင့် tool အသုံးပြုမှုအသေးစိတ်ကို ပြသပါ။

**Keyless အင်္ဂါရပ်များ**:
- ✅ **လျှို့ဝှက်ချက်များ မလိုအပ်ခြင်း** - Authentication အားလုံးကို Azure identity ဖြင့် ကိုင်တွယ်ထားသည်။
- ✅ **ပုံမှန်အားဖြင့် လုံခြုံမှုရှိခြင်း** - Role-based access control ကို အသုံးပြုထားသည်။
- ✅ **တပ်ဆင်မှု လွယ်ကူခြင်း** - Credential ကို စီမံခန့်ခွဲရန် မလိုအပ်ပါ။
- ✅ **စစ်ဆေးမှုအတွက် သင့်လျော်မှုရှိခြင်း** - Access အားလုံးကို Azure identity ဖြင့် မှတ်တမ်းတင်ထားသည်။

အေးဂျင့်သည် MCP tools များကို အသုံးပြု၍ Microsoft Learn ရင်းမြစ်များကို လုံခြုံစိတ်ချစွာ ဝင်ရောက်အသုံးပြုမည်ဖြစ်ပြီး API key စီမံခန့်ခွဲမှုမလိုအပ်ပါ။


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါရှိနိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာရှိသော ရင်းမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအမှားများ သို့မဟုတ် အနားလွဲမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
